In [1]:
from PPOAgent import PPOAgent
from SubEnvironment import SubEnvironment
from ParallelEnvironment import ParallelEnvironment
import time
import gym
%load_ext autoreload
%autoreload 2

In [2]:
import gym
envs = [lambda: gym.make('CartPole-v0') for x in range(2)]

In [16]:
for i in range(10):
    start_time = time.time()
    envs = [lambda: gym.make('CartPole-v0') for x in range(2)]
    venv = ParallelEnvironment(envs)
    agent = PPOAgent(venv, buffer_size=64)
    agent.run(50)
    print("{:.3f}".format(time.time() - start_time))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
16.802
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
14.592
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
22.527
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
20.491
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provid

OSError: [Errno 12] Cannot allocate memory

In [ ]:
for i in range(10):
    start_time = time.time()
    envs = [lambda: gym.make('CartPole-v0') for x in range(1)]
    venv = ParallelEnvironment(envs)
    agent = PPOAgent(venv, buffer_size=64)
    agent.run(50)
    print("{:.3f}".format(time.time() - start_time))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
env = gym.make('CartPole-v0')
agent = PPOAgent(env, verbose=1)
agent.run(50)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Process Process-3:
Traceback (most recent call last):
  File "/home/nathan/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/nathan/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nathan/Agent-Resource-Allocation/ParallelEnvironment.py", line 21, in _worker
    env = env_fn_wrapper.var()
TypeError: 'TimeLimit' object is not callable


ConnectionResetError: [Errno 104] Connection reset by peer

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
from sidecar import Sidecar

In [ ]:
run_dir = "./runs/agent-27/data/"
line_limit = 10
log_path = run_dir + 'agent.txt'
log_val = []
kill_thread = False
last_frame_count = 0
empty_frame_count = 0

def update_log():
    log_val = ['<li><code style="font-size: 10px;">{}</code></li>'.format(line) for 
               line in list(open(log_path))[-line_limit:]]
    log_widget.value = '<ul style="list-style-type: none; padding: 0;">' + ''.join(log_val) + '</ul>'
    
def on_change(x):
    if (x['name'] == 'value'):
        update_image(x['new'])
        
def kill_thread_click(x):
    global kill_thread
    kill_thread = True
        
def live_update():
    global last_frame_count
    global empty_frame_count
    while True:
        update_count_widget.value = "Last Update: " + time.strftime('%c')
        update_log()
        if last_frame_count == frame_count:
            empty_frame_count += 1
        else:
            last_frame_count = frame_count
            empty_frame_count = 0
        if empty_frame_count > 10 or kill_thread:
            update_count_widget.value = 'Halted: ' + time.strftime('%c')
            break
        time.sleep(1)

play_widget = widgets.Play(
    interval=250,
    value=frame_count,
    min=0,
    max=frame_count,
    step=1,
    description="Press play",
    disabled=False
)
slider_widget = widgets.IntSlider(max=frame_count)
image_widget = widgets.Image(format='png', height=100)
 
slider_widget.observe(on_change)
widgets.jslink((play_widget, 'value'), (slider_widget, 'value'))

log_widget = widgets.HTML()

update_count_widget = widgets.Label()

kill_button_widget = widgets.Button(
    description='Kill Thread'
)
kill_button_widget.on_click(kill_thread_click)

thread = threading.Thread(target=live_update)
thread.start()

training_info = widgets.VBox([image_widget,
                      widgets.HBox([kill_button_widget, play_widget, slider_widget, update_count_widget]),
                      log_widget])

sc = Sidecar(title='Training Info')
with sc:
    display(training_info)

## ZeroMQ

In [ ]:
import zmq

In [ ]:
context = zmq.Context()
with context.socket(zmq.REQ) as socket:
    socket.connect('tcp://127.0.0.1:5555')
    socket.send_string('zeromq')
    msg = socket.recv()
    print(msg)